In [1]:
import os

import dask
from dask_jobqueue import PBSCluster
from dask.distributed import Client

from fates_calibration.post_process_LH_ensemble import postprocess_files
from fates_calibration.compile_LH_ensemble import compile_ensemble

In [2]:
# Setup PBSCluster
cluster = PBSCluster(
    cores=1,                                     # The number of cores you want
    memory='25GB',                               # Amount of memory
    processes=1,                                 # How many processes
    queue='casper',                              # The type of queue to utilize
    local_directory='/glade/work/afoster',       # Use your local directory
    resource_spec='select=1:ncpus=1:mem=25GB',   # Specify resources]
    log_directory='/glade/derecho/scratch/afoster/dask_logs',
    account='P08010000',                         # Input your project ID here
    walltime='02:00:00',                         # Amount of wall time
    interface='ext')                             # Interface to use

cluster.scale(20)
dask.config.set({
    'distributed.dashboard.link': 'https://jupyterhub.hpc.ucar.edu/stable/user/{USER}/proxy/{port}/status'
})
client = Client(cluster)
client

/glade/work/afoster/conda-envs/fates_calibration/lib/python3.11/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 37171 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/afoster/proxy/37171/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/afoster/proxy/37171/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://128.117.208.81:40611,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/afoster/proxy/37171/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [3]:
archive_dir = '/glade/derecho/scratch/afoster/transient_ensemble/spinup/archive'
postp_dir = '/glade/work/afoster/FATES_calibration/old_calib/FATES_SP_LH/transient_hist_co2'
out_dir = '/glade/work/afoster/FATES_calibration/old_calib/FATES_SP_LH/'
out_file = 'transient_hist_co2.nc'

In [4]:
# postprocess the individual jobs
postprocess_files(archive_dir, postp_dir)

All ensembles passed!


In [ ]:
# combine into one file, calculate some output variables
out_file = os.path.join(out_dir, out_file)
compile_ensemble(postp_dir, out_file)

In [ ]:
client.shutdown()